# Preparation

In [4]:
import math
import os
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
sys.path.insert(0,'../../../../') # The simulator package and package needed for analysis 
                                  # (which are imported below) are in a parent folder

# Modified package from https://github.com/BilHim/trafficSimulator
from trafficSimulator import *
# Functions needed for simulation transition calculation
from SimulationAnalysis import utility_summary, compute_utility, update_utility_df

In [ ]:
vehicle_rate = 45

## Simulation setup

### Simulation params setup

In [5]:
# Specify the labels of the road segments in each route
all_routes = [[0, 3], [0, 1], [2, 3], [2, 1]]

v_max = 9
s0 = 2
T = 1
b_max = 2
a_max = 1

# Levels of speed limit for fast and slow roads
# expressed as a proportion of the v_max set above
fast_track_factor, slow_track_factor = 1, 0.33 

# Length of the stop zone
stop_distance = 25 

# Waiting time for fast and slow roads (here set as the same)
fast_wait_time = 4
slow_wait_time = 4

# Total number of vehicles
vehicle_limit = 80

# Store vehicle specs to be input in the simulation package
vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                 'l' : 3
                }

# Empty dataframe to store the simulation data
df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

# Initialize the probabilities for choosing each route for the 1st round;
# each route has an equal chance of being selected
vehicle_preferences = dict(zip(range(vehicle_limit), 
                               [[0.25, 0.25, 0.25, 0.25]]*vehicle_limit))

### Topology setup

In [7]:
# Coordinates of the four corners of the traffic network; all four corners
# lie on either the x or the y axis
left, right = -75*math.sqrt(3), 75*math.sqrt(3)
bottom, top = -75, 75

# Building the road lines with corresponding parameters
left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

## For the connections of the topology, we can represent with concrete lines, but
# the vehicles will not actually go through these routes; 
# they are  for demonstration purposes
connection_top_bottom = ((-1.25, bottom+2), (-1.25, top-2))
connection_bottom_top = ((1.25, top-2), (1.25, bottom+2))

### Utility params setup

In [8]:
# Exploration factor in UCB alogorithm
delta = 0.25

# Penalty factor
alpha = 2.5

## Function needed for running simulation

In [9]:
 def run_simulation(round_number, 
                    all_routes, 
                    vehicle_limit, 
                    df, 
                    vehicle_preferences,
                    vehicle_rate, 
                    vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    global connection_top_bottom
    global connection_bottom_top
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        connection_top_bottom, # Road #4
        connection_bottom_top, # Road #5

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go pass these
        # routes; they are more for aesthetic purposes
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 4
    new_records = win.run(steps_per_update=5)
    
    return new_records

# Official start: 1st round

In [10]:
%%time
# Run the 1st round
record_df1 = run_simulation(1, all_routes, vehicle_limit, df, vehicle_preferences,
                             vehicle_rate, vehicle_specs)

CPU times: user 3min 7s, sys: 4.38 s, total: 3min 12s
Wall time: 3min 11s


In [11]:
record_df1

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front
0,0,"[2, 3]",67.8,"[-999, -999]","[0, 0]","[0, 0]"
1,1,"[0, 3]",102.45,"[-999, 12]","[0, 0]","[0, 0]"
2,2,"[2, 1]",37.65,"[0, -999]","[0.49999999999999994, 0]","[0, 0]"
3,3,"[2, 1]",41.6,"[2, 2]","[2.6999999999999984, 0]","[0, 0]"
4,4,"[0, 3]",103.55,"[1, 1]","[0.05, 0]","[0, 0]"
...,...,...,...,...,...,...
75,75,"[0, 1]",166.0,"[74, 60]","[36.300000000000075, 0]","[0, 0]"
76,76,"[2, 3]",247.05,"[73, 73]","[86.9499999999972, 0]","[0, 0]"
77,77,"[2, 1]",219.9,"[76, -999]","[90.04999999999703, 0]","[0, 0]"
78,78,"[2, 3]",255.0,"[77, 76]","[93.09999999999685, 0]","[0, 0]"


## Simulation Analysis of the 1st round

In [12]:
record_df1['Round_number'] = 1

record_df = record_df1.copy()
record_df.head(10)

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front,Round_number
0,0,"[2, 3]",67.8,"[-999, -999]","[0, 0]","[0, 0]",1
1,1,"[0, 3]",102.45,"[-999, 12]","[0, 0]","[0, 0]",1
2,2,"[2, 1]",37.65,"[0, -999]","[0.49999999999999994, 0]","[0, 0]",1
3,3,"[2, 1]",41.6,"[2, 2]","[2.6999999999999984, 0]","[0, 0]",1
4,4,"[0, 3]",103.55,"[1, 1]","[0.05, 0]","[0, 0]",1
5,5,"[2, 1]",44.35,"[3, 3]","[3.599999999999995, 0]","[0, 0]",1
6,6,"[2, 1]",48.35,"[5, 5]","[5.799999999999987, 0]","[0, 0]",1
7,7,"[2, 1]",52.15,"[6, 6]","[8.04999999999998, 0]","[0, 0]",1
8,8,"[2, 3]",81.45,"[7, 0]","[10.300000000000011, 0]","[0, 0]",1
9,9,"[0, 3]",101.75,"[4, 4]","[0.05, 0]","[0, 0]",1


### Set up dataframe

In [13]:
utility_df = pd.read_csv("../2.3.2.1 Zero alpha/Saved_data/Utility_data/Round_1.csv")

utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
utility_df['Routes_taken'] = [[]] * len(utility_df)
utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)

utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            
utility_df.head(5)

,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],5.310139,0.517878
1,1,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],4.825830,0.568200
2,2,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],4.590759,0.308298
3,3,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],5.157208,0.564622
4,4,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],5.026788,0.383511


## Summary of the 1st round

In [14]:
utility_tmp = utility_summary(all_routes, record_df)
utility_df = update_utility_df(all_routes,
                               delta, 
                               alpha, 
                               record_df=record_df,
                               df=utility_df, 
                               utility_tmp=utility_tmp)

display(utility_tmp.head(10))
display(utility_df.head(10))

,Vehicle_label,Road_order,Caused_delay,Count,Total_time
0,0,2,0.50,1,67.80
1,1,0,0.05,1,102.45
2,2,3,2.70,1,37.65
3,3,3,3.60,1,41.60
4,4,0,0.05,1,103.55
5,5,3,5.80,1,44.35
6,6,3,8.05,1,48.35
7,7,3,10.30,1,52.15
8,8,2,8.45,1,81.45
9,9,0,2.00,1,101.75


,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[16.651092223153956, 16.651092223153956, -0.49...","[0.3333333293751783, 0.3333333293751783, 1.187...",[2],5.310139,0.517878
1,1,"[2.876571386163249, 16.651092223153956, 16.651...","[3.472803448547161e-07, 0.3333332175732184, 0....",[0],4.825830,0.568200
2,2,"[16.651092223153956, 16.651092223153956, 16.65...","[0.3333333304019605, 0.3333333304019605, 0.333...",[3],4.590759,0.308298
3,3,"[16.651092223153956, 16.651092223153956, 16.65...","[0.33333333228141615, 0.33333333228141615, 0.3...",[3],5.157208,0.564622
4,4,"[4.300715296483071, 16.651092223153956, 16.651...","[1.4427051099796501e-06, 0.33333285243162997, ...",[0],5.026788,0.383511
5,5,"[16.651092223153956, 16.651092223153956, 16.65...","[0.3333333318968758, 0.3333333318968758, 0.333...",[3],4.988453,0.462755
6,6,"[16.651092223153956, 16.651092223153956, 16.65...","[0.3333333322431772, 0.3333333322431772, 0.333...",[3],4.878581,0.506162
7,7,"[16.651092223153956, 16.651092223153956, 16.65...","[0.3333333326133111, 0.3333333326133111, 0.333...",[3],4.805346,0.571507
8,8,"[16.651092223153956, 16.651092223153956, -1.94...","[0.3333333324052334, 0.3333333324052334, 2.784...",[2],4.348500,0.530272
9,9,"[-1.0507923443998772, 16.651092223153956, 16.6...","[6.8398716981942705e-09, 0.3333333310533761, 0...",[0],5.786703,0.416448


In [15]:
# Export
record_df.to_csv("Saved_data/Simulation_records/Round_1.csv", index=False)
utility_df.to_csv("Saved_data/Utility_data/Round_1.csv", index=False)

# Run the remaining rounds
Utilities, probabilities are updated after each round and fed into the next round

In [16]:
%%time

# Updated probabilities for the 2nd round:
vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))

rounds = 50

for s in range(2, rounds+1):
    record_tmp = run_simulation(s, 
                                all_routes, 
                                vehicle_limit, 
                                df, 
                                vehicle_preferences,
                                vehicle_rate, 
                                vehicle_specs)
    record_tmp['Round_number'] = s
    
    record_df = pd.concat([record_df, record_tmp], ignore_index=True)
    
    utility_tmp = utility_summary(all_routes, record_df)
    utility_df = update_utility_df(all_routes,
                                   delta, 
                                   alpha, 
                                   record_df=record_df,
                                   df=utility_df, 
                                   utility_tmp=utility_tmp,
                                   round_number=s)
    
    vehicle_preferences = dict(zip(range(vehicle_limit), 
                                   utility_df.Probabilities.to_list()))
    
    # Export
    record_df.to_csv(f"Saved_data/Simulation_records/Round_{s}.csv", index=False)
    utility_df.to_csv(f"Saved_data/Utility_data/Round_{s}.csv", index=False)

CPU times: user 2h 38min 12s, sys: 4min 8s, total: 2h 42min 20s
Wall time: 2h 36min 34s
